### Plotly chart wrappers (python)
  
This is a set of functions wrapping the powerful customization options of Plotly charts into single-function calls with a few parameters that should produce good-looking charts covering at least 50% of the typical charting of a data analyst. The rationale behind writing the wrappers is twofold:  
1) To streamline analyst work and make the generation of lucid charts accessible even to analysts who are not familiar with the intricacies of Plotly's Python library  
2) To set up a consistent visual style that would be easily customizable to fit any corporate design by pre-defining colors and font styles used in all the charts

In [1]:
import pandas as pd
import numpy as np

import plotly.offline as py
from plotly.graph_objs import *
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
%matplotlib inline

In [2]:
dat = pd.read_csv("https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-Marketing-Campaign-Eff-UseC_-FastF.csv")

In [3]:
dat.head()

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81
